In [2]:
import os
from langchain_openai import ChatOpenAI
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(
    temperature=0,
    openai_api_key=os.getenv("TOGETHER_API_KEY"),
    openai_api_base="https://api.together.xyz/v1",
    model="meta-llama/Llama-3-70b-chat-hf"
)

result = llm.invoke("What is LangChain in machine learning?")
print(result.content)


LangChain is a type of artificial neural network architecture that combines the strengths of large language models (LLMs) with the power of chain-of-thought reasoning. It's a relatively new concept in machine learning, introduced in 2022, and has been gaining attention in the research community.

**The idea behind LangChain:**

LangChain is designed to address the limitations of traditional language models, which are excellent at generating text but often struggle with complex, multi-step reasoning tasks. The architecture combines a large language model (LLM) with a chain-of-thought reasoning module, which allows the model to break down complex problems into a series of simpler, more manageable steps.

**How LangChain works:**

1. **Language Model (LLM):** The LLM generates an initial response to a given prompt or question. This response is often incomplete or inaccurate, but it provides a starting point for the chain-of-thought reasoning process.
2. **Chain-of-Thought Reasoning:** The

In [3]:
import os
from langchain.chat_models import ChatOpenAI
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter

def initialize_llm():
    llm = ChatOpenAI(
        temperature=0,
        openai_api_key=os.getenv("TOGETHER_API_KEY"),  # safer: use env variable
        openai_api_base="https://api.together.xyz/v1",
        model="meta-llama/Llama-3-70b-chat-hf"
    )
    return llm

def create_vector_db():
    loader = DirectoryLoader(
        "data",
        glob='*.pdf',
        loader_cls=PyPDFLoader
    )
    documents = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    texts = text_splitter.split_documents(documents)
    embeddings = HuggingFaceEmbeddings(
        model_name='sentence-transformers/all-MiniLM-L6-v2'
    )
    vector_db = Chroma.from_documents(
        texts, 
        embeddings, 
        persist_directory='./chroma_db'
    )
    print("Chroma DB created and data saved")
    return vector_db

def setup_qa_chain(vector_db, llm):
    retriever = vector_db.as_retriever()
    prompt = PromptTemplate(
        input_variables=["context", "question"],
        template="Use the following context to answer the question.\nContext: {context}\nQuestion: {question}\nAnswer:"
    )
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=retriever,
        return_source_documents=True,
        chain_type_kwargs={"prompt": prompt}
    )
    return qa_chain

def main():
    print("Initializing ChatBot...")
    llm = initialize_llm()

    db_path = "chroma_db"

    if not os.path.exists(db_path):
        vector_db = create_vector_db()
    else:
        vector_db = Chroma(persist_directory=db_path, embedding_function=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2'))

    qa_chain = setup_qa_chain(vector_db, llm)

    while True:
        user_input = input("Enter your question: ")
        if user_input.lower() == "exit":
            print("ChatBot: Take care of yourself. Goodbye!")
            break

        result = qa_chain({"query": user_input})
        print("ChatBot:", result.get("result", ""))

if __name__ == "__main__":
    main()


Initializing ChatBot...


C:\Users\ENG CHANDRAKANT\AppData\Local\Temp\ipykernel_28648\2036458690.py:28: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


Chroma DB created and data saved


C:\Users\ENG CHANDRAKANT\AppData\Local\Temp\ipykernel_28648\2036458690.py:73: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain({"query": user_input})


ChatBot: Based on the context, it's not possible to determine if you have a diagnosable disorder of depression or if you're simply experiencing feelings of unhappiness, despondency, frustration, and hopelessness associated with a state of emotional stress. Depression is a diagnosable disorder that involves pervasive physical symptoms such as sleep and appetite disturbance, and changes in brain chemistry. If you're experiencing persistent feelings of sadness, hopelessness, and a lack of interest in activities you once enjoyed, it's recommended that you seek professional help from a mental health expert for a proper evaluation and diagnosis.
ChatBot: Based on the context provided, it's important to clarify that feeling depressed and having depression are not exactly the same thing. Depression is a diagnosable disorder characterized by pervasive physical symptoms such as sleep and appetite disturbance, and changes in brain chemistry. 

Feeling depressed, on the other hand, can be a normal

In [4]:
import gradio as gr
from langchain.document_loaders import DirectoryLoader, PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from dotenv import load_dotenv

load_dotenv()  # Loads from .env file

def initialize_llm():
    from dotenv import load_dotenv
    load_dotenv()
    
    together_api_key = os.getenv("TOGETHER_API_KEY")  # Load from .env file
    if not together_api_key:
        raise ValueError("TOGETHER_API_KEY not found in environment variables")

    llm = ChatOpenAI(
        temperature=0.7,
        openai_api_key=together_api_key,
        openai_api_base="https://api.together.xyz/v1",
        model="meta-llama/Llama-3-70b-chat-hf"
    )
    return llm

# Create vector database function
def create_vector_db():
    loader = DirectoryLoader(
        "data",
        glob='*.pdf',
        loader_cls=PyPDFLoader
    )
    documents = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    texts = text_splitter.split_documents(documents)
    embeddings = HuggingFaceEmbeddings(
        model_name='sentence-transformers/all-MiniLM-L6-v2'
    )
    vector_db = Chroma.from_documents(
        texts, 
        embeddings, 
        persist_directory='./chroma_db'
    )
    print("Chroma DB created and data saved")
    return vector_db

# Setup QA chain function
def setup_qa_chain(vector_db, llm):
    retriever = vector_db.as_retriever()
    prompt = PromptTemplate(
        input_variables=["context", "question"],
        template="""
        Respond compassionately to emotional queries. Provide helpful information.
        If the question indicates distress, offer support resources.
        
        Context: {context}
        Question: {question}
        
        Answer:
        """
    )
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=retriever,
        return_source_documents=True,
        chain_type_kwargs={"prompt": prompt}
    )
    return qa_chain

# Initialize components
print("Initializing ChatBot...")
llm = initialize_llm()

db_path = "chroma_db"
if not os.path.exists(db_path):
    vector_db = create_vector_db()
else:
    vector_db = Chroma(
        persist_directory=db_path,
        embedding_function=HuggingFaceEmbeddings(
            model_name='sentence-transformers/all-MiniLM-L6-v2'
        )
    )

qa_chain = setup_qa_chain(vector_db, llm)

# Modern Mental Health CSS
custom_css = """
@import url('https://fonts.googleapis.com/css2?family=Nunito:wght@300;400;600;700&display=swap');

:root {
    --primary-gradient: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
    --secondary-gradient: linear-gradient(135deg, #f093fb 0%, #f5576c 100%); 
    --healing-gradient: linear-gradient(135deg, #4facfe 0%, #00f2fe 100%);
    --warm-gradient: linear-gradient(135deg, #fa709a 0%, #fee140 100%);
    --glass-bg: rgba(255, 255, 255, 0.25);
    --glass-border: rgba(255, 255, 255, 0.3);
    --text-primary: #2d3748;
    --text-light: #4a5568;
    --shadow-soft: 0 8px 32px rgba(31, 38, 135, 0.37);
    --shadow-glow: 0 0 30px rgba(102, 126, 234, 0.3);
}

* {
    font-family: 'Nunito', sans-serif !important;
}

.gradio-container {
    background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
    min-height: 100vh;
    position: relative;
}

/* Remove unnecessary animations */
@keyframes float {
    0%, 100% { transform: translateY(0px); }
    50% { transform: translateY(-5px); }
}

@keyframes pulse {
    0%, 100% { opacity: 1; }
    50% { opacity: 0.8; }
}

.chatbot {
    background: var(--glass-bg) !important;
    backdrop-filter: blur(20px) !important;
    border: 1px solid var(--glass-border) !important;
    border-radius: 24px !important;
    box-shadow: var(--shadow-soft) !important;
    overflow: hidden !important;
    position: relative !important;
}

.chatbot::before {
    content: '';
    position: absolute;
    top: 0;
    left: 0;
    right: 0;
    height: 4px;
    background: var(--healing-gradient);
    z-index: 1;
}

.message.user {
    background: var(--primary-gradient) !important;
    color: white !important;
    border-radius: 20px 20px 4px 20px !important;
    margin-left: auto !important;
    max-width: 75% !important;
    padding: 16px 20px !important;
    font-weight: 500 !important;
    box-shadow: 0 4px 15px rgba(102, 126, 234, 0.4) !important;
    border: none !important;
    position: relative !important;
    display: flex !important;
    align-items: center !important;
    gap: 12px !important;
}

.message.bot {
    background: rgba(255, 255, 255, 0.98) !important;
    color: #1a202c !important;
    border-radius: 20px 20px 20px 4px !important;
    box-shadow: 0 4px 20px rgba(0, 0, 0, 0.2) !important;
    max-width: 75% !important;
    padding: 16px 20px !important;
    border: 1px solid rgba(255, 255, 255, 0.6) !important;
    line-height: 1.6 !important;
    position: relative !important;
    backdrop-filter: blur(10px) !important;
    display: flex !important;
    align-items: flex-start !important;
    gap: 12px !important;
}

/* Ensure all chatbot content is visible */
.chatbot .message.bot,
.chatbot .message.bot p,
.chatbot .message.bot div,
.chatbot .message.bot span,
.chatbot .message.bot ul,
.chatbot .message.bot li,
.chatbot .message.bot strong,
.chatbot .message.bot em {
    color: #1a202c !important;
    font-weight: 500 !important;
}

.chatbot .message.bot {
    background: white !important;
    border: 2px solid rgba(102, 126, 234, 0.2) !important;
}

.message.bot p {
    margin-bottom: 12px !important;
    color: #1a202c !important;
    font-weight: 500 !important;
}

.message.bot ul {
    padding-left: 20px !important;
    color: #2d3748 !important;
}

.message.bot li {
    margin-bottom: 6px !important;
    color: #2d3748 !important;
}

.message.bot * {
    color: #1a202c !important;
}

/* Input styling */
.input-container {
    background: var(--glass-bg) !important;
    backdrop-filter: blur(15px) !important;
    border-radius: 20px !important;
    border: 1px solid var(--glass-border) !important;
    box-shadow: var(--shadow-soft) !important;
}

textarea {
    background: transparent !important;
    color: white !important;
    border: none !important;
    font-size: 16px !important;
    font-weight: 400 !important;
    resize: none !important;
}

textarea::placeholder {
    color: rgba(255, 255, 255, 0.7) !important;
}

/* Button styling */
button {
    background: var(--healing-gradient) !important;
    border: none !important;
    border-radius: 12px !important;
    color: white !important;
    font-weight: 600 !important;
    padding: 12px 24px !important;
    transition: all 0.3s ease !important;
    box-shadow: 0 4px 15px rgba(79, 172, 254, 0.4) !important;
}

button:hover {
    transform: translateY(-2px) !important;
    box-shadow: 0 6px 25px rgba(79, 172, 254, 0.6) !important;
}

/* Example buttons */
.examples button {
    background: rgba(255, 255, 255, 0.1) !important;
    border: 1px solid rgba(255, 255, 255, 0.2) !important;
    border-radius: 16px !important;
    color: white !important;
    padding: 10px 16px !important;
    margin: 4px !important;
    font-size: 14px !important;
    backdrop-filter: blur(10px) !important;
    transition: all 0.3s ease !important;
}

.examples button:hover {
    background: rgba(255, 255, 255, 0.2) !important;
    transform: translateY(-1px) !important;
    box-shadow: 0 4px 15px rgba(255, 255, 255, 0.1) !important;
}

/* Avatar styling */
.avatar {
    flex-shrink: 0 !important;
    width: 32px !important;
    height: 32px !important;
    border-radius: 50% !important;
    overflow: hidden !important;
    border: 2px solid rgba(255, 255, 255, 0.8) !important;
    box-shadow: 0 2px 8px rgba(0, 0, 0, 0.2) !important;
}

.avatar img {
    width: 100% !important;
    height: 100% !important;
    object-fit: cover !important;
    border-radius: 50% !important;
}

/* Header styling */
h1 {
    color: white !important;
    text-shadow: 0 2px 10px rgba(0, 0, 0, 0.3) !important;
    font-weight: 700 !important;
    font-size: 2.5rem !important;
}

p {
    color: rgba(255, 255, 255, 0.9) !important;
    text-shadow: 0 1px 3px rgba(0, 0, 0, 0.3) !important;
    font-weight: 400 !important;
    font-size: 1.1rem !important;
}

/* Remove footer */
footer {
    display: none !important;
}

/* Scrollbar styling */
::-webkit-scrollbar {
    width: 8px;
}

::-webkit-scrollbar-track {
    background: rgba(255, 255, 255, 0.1);
    border-radius: 4px;
}

::-webkit-scrollbar-thumb {
    background: var(--healing-gradient);
    border-radius: 4px;
}

::-webkit-scrollbar-thumb:hover {
    background: var(--primary-gradient);
}

/* Loading animation */
.loading {
    position: relative;
}

.loading::after {
    content: '';
    position: absolute;
    top: 50%;
    left: 50%;
    width: 20px;
    height: 20px;
    margin: -10px 0 0 -10px;
    border: 2px solid rgba(79, 172, 254, 0.3);
    border-top: 2px solid #4facfe;
    border-radius: 50%;
    animation: spin 1s linear infinite;
}

@keyframes spin {
    0% { transform: rotate(0deg); }
    100% { transform: rotate(360deg); }
}

/* Mobile responsiveness */
@media (max-width: 768px) {
    .message.user, .message.bot {
        max-width: 90% !important;
        padding: 12px 16px !important;
    }
    
    h1 {
        font-size: 2rem !important;
    }
    
    .chatbot {
        border-radius: 16px !important;
        margin: 10px !important;
    }
}
"""

def chatbot_response(message, history):
    if not message.strip():
        return "Please enter a valid message"
    
    try:
        result = qa_chain({"query": message})
        response = result.get("result", "I couldn't generate a response.")
        
        if any(word in message.lower() for word in ["depressed", "sad", "suicide", "anxious"]):
            response += """
            \n\n💜 If you're in distress, please consider contacting:
            - National Suicide Prevention Lifeline: 988 (US)
            - Crisis Text Line: Text HOME to 741741 (US)
            """
            
        return response
        
    except Exception as e:
        print(f"Error: {str(e)}")
        return "Sorry, I encountered an error processing your request."

# Create Gradio interface with modern anime theme
with gr.Blocks(css=custom_css, theme=gr.themes.Default(primary_hue="violet")) as app:
    # Set favicon via HTML head
    app.title = "Anime Mental Health Companion"
    app.head = """<link rel="icon" href="https://i.imgur.com/3L8JQ2x.png">"""
    
    with gr.Row():
        gr.Markdown("""
        <div style="text-align: center; margin-bottom: 30px;">
            <h1>🌸 Mental Health Companion 🌸</h1>
            <p>Your supportive AI wellness companion</p>
        </div>
        """)
    
    # Using the modern ChatInterface format
    chat_interface = gr.ChatInterface(
        fn=chatbot_response,
        examples=[
            ["I'm feeling really anxious today 😰"],
            ["How can I practice mindfulness?"],
            ["What are some self-care tips?"],
            ["I'm having trouble sleeping"],
            ["How do I deal with stress?"]
        ],
        additional_inputs=None,
        title="",
        description=""
    )
    
    # Workaround for avatars in newer Gradio versions
    chat_interface.chatbot.avatar_images = [
        ("https://i.imgur.com/7WqjD9T.png", "user"),
        ("https://i.imgur.com/3L8JQ2x.png", "bot")
    ]

# Launch without deprecated parameters
app.launch(share=True)

Initializing ChatBot...


C:\Users\ENG CHANDRAKANT\AppData\Local\Temp\ipykernel_28648\1700259008.py:81: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vector_db = Chroma(
c:\Users\ENG CHANDRAKANT\anaconda3\Lib\site-packages\gradio\chat_interface.py:339: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://244822a61179cab982.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
